In [1]:
import numpy as np
import scipy.optimize as opt

**Exercise 9.6**

In [2]:
def steep_desc(Q, x_0, b, tol=1e-10, max_iter=3000):
    '''Implements the steppest descent method for quadratic functions
    (see Example 9.2.3 in book)
    '''
    d = 1
    k = 1
    while k < max_iter and d > tol:
        Df = (Q @ x_0) - b
        α = (Df.T @ Df) / ((Df.T @ Q) @ Df) # see equation 9.2 in book
        d = np.linalg.norm(Df)
        x_1 = x_0 - α * Df
        x_0 = x_1
    
    if k < max_iter:
        print("Convergence achieved!")
    
    return x_0

In [3]:
Q = np.array([[5,-2],[-2,5]])
x_0 = np.array([3,3])
b = np.array([4,2])

steep_desc(Q, x_0, b)

Convergence achieved!


array([1.14285714, 0.85714286])

**Exercise 9.7**

In [4]:
#def f(x):
#    '''Defines a function f in quadratic form, with input x
#    '''
#    return 0.5 * x @ (Q @ x) - b @ x + 4
 
#f = lambda x: 0.5 * x @ (Q @ x) - b @ x + 4

def compute_Df(f, x_0, Rerr_f):
    '''Computes Df using forward differences and a step size of
    sqrt(Rerr_f). 
    Accepts a callable function f:R^n -> R, a point x in R^n, and
    an estimate Rerr_f > varepsilon_machine for the maximum relative
    error of f near x.
    Returns an estimate for Df(x).
    See page 423 of notes.
    '''
    
    # Old code
    """
    m = f(x_0).shape
    if len(m) == 0:
        m = 1
    n = x_0.shape[0] # x_0 in R^n
    
    Df = np.zeros((m,n))
    h = 2 * np.sqrt(Rerr_f) # Approximately
    
    for i in range(n):
        e_i = np.zeros(n)
        e_i[i] = 1 # Standard basis
        Df[:,i] = (f(x_0 + h * e_i) - f(x_0)) * (1 / h)
    
    return Df
    """
    
    n = len(x_0)
    Df = np.zeros(n, dtype=np.float64)
    h = 2 * np.sqrt(Rerr_f)
    for i in range(n):
        ei = np.eye(n)[:,i]
        Df[i] = (-3 * f(x_0) + 4 * f(x_0 + h * ei) - \
                 f(x_0 + 2 * h * ei)) / (2 * h)
    return Df

**Exercise 9.8**

In [5]:
def secant(x, x_1, ɛ, fp):
    '''Implements the secant method
    '''
    
    max_iter = 2000
    iterate = 1
    
    x_km1 = x
    x_k = x_1
    
    converged = False
    while not converged and iterate < max_iter:
        
        update = fp(x_k) * ((x_k - x_km1) / (fp(x_k) - fp(x_km1)))
        x_k1 = x_k - update
                               
        if np.abs(x_k1 - x_k) < np.abs(x_k) * ɛ:
            converged = True
        else: 
            converged = False
        
        x_km1 = x_k
        x_k = x_k1
        
        iterate += 1
    
    return x_k1

In [6]:
def steep_desc2(f, x, ɛ):
    '''Implements the steepest descent method for arbitrary 
    functions.
    '''
    
    max_iter = 20000
    iterate = 1
    error = 1 + ɛ
    
    x_k = x
    
    while error > ɛ and iterate < max_iter:
        
        Dfx = compute_Df(f, x_k, ɛ)
        fαp = lambda x: compute_Df(f, x_k - x * Dfx.T, ɛ) @ (-1 * Dfx.T)
        αopt = secant(.8, .2, ɛ, fαp)
        x_kp1 = x_k - αopt * Dfx
        error = np.linalg.norm(Dfx)
        x_k = x_kp1
        iterate += 1
    
    return x_k, error

**Exercise 9.9**

In [7]:
def rosenbrock(x):
    return 100 * (x[1] - x[0] ** 2) ** 2 + (1 - x[0]) ** 2

In [8]:
x = np.array([-2, 2])

In [9]:
y = steep_desc2(rosenbrock, x, 1e-5)
y

(array([1.01634609, 1.03295939]), 9.999155640197579e-06)